In [ ]:
import numpy as np
import plotly.graph_objects as go
import pandas as pd
import plotly.express as px
from EC_data_processing_lib import get_header_length, get_decimal_separator
from pandas import DataFrame as df
from Data_setup_CHEM import data_set  # choose the correct python file
import os
from galvani import BioLogic
from datetime import datetime

fig_title = 'Title'
today_str = datetime.today().strftime('%Y-%m-%d')
fig = go.Figure()
colors = px.colors.qualitative.Dark24

for i in data_set:
    file = data_set[i]['data_CP']
    if not os.path.exists(file):
        print("File not found: " + file)
        continue
    if file.endswith('mpt'):
        headerlength = get_header_length(file)
        data = pd.read_csv(
            file,
            encoding='ansi',
            sep="\t",
            decimal=get_decimal_separator(file),
            skiprows=range(headerlength)
        )
    else:
        mpr_file = BioLogic.MPRfile(file)
        data = pd.DataFrame(mpr_file.data)
    
    # Automatically select the correct current column
    if '<I>/mA' in data.columns:
        current_column = '<I>/mA'
    elif 'I/mA' in data.columns:
        current_column = 'I/mA'
    else:
        print("No current column found in file: " + file)
        continue

    voltage = data['<Ewe>/V']
    time = data['time/s'] / 60  # convert seconds to minutes
    time = time - time.iloc[0]  # set initial time to 0
    endtime = time.iloc[-1]
    print("End time (min):", endtime)
    
    # Apply iR correction using the selected current column.
    voltage_corrected = data['<Ewe>/V'] - data[current_column] * data_set[i]['Ru'] / 1000 - data_set[i]['cal']
    voltage_corrected = voltage_corrected * 1000  # convert to mV

    # Calculate final overpotential values (for time > 115 min) and print them (optional)
    Final_Values = []
    if any(time > 115):
        final_eta = df.mean(voltage_corrected)
        final_eta = np.round(final_eta)
        final_eta_error = df.std(voltage_corrected)
        final_eta_error = np.round(final_eta_error)
        Final_Values = [data_set[i]['label'], float(final_eta), float(final_eta_error)]
        Final_Values = np.array(Final_Values)
        print(Final_Values)

    fig.add_trace(go.Scatter(
        x=time,
        y=voltage_corrected,
        name=data_set[i]['label'],
        line_color=colors[data_set[i]['color_index']],
    ))

# Update x-axis and y-axis with styling and autorange
fig.update_xaxes(
    range=[0, 120],
    nticks=10,
    ticks='outside',
    showgrid=False,
    gridwidth=1,
    gridcolor='grey',
    zeroline=False,
    zerolinecolor='grey',
    zerolinewidth=1,
    showline=True,
    linewidth=2,
    linecolor='black'
)

fig.update_yaxes(
    range=[300, 2000],
    nticks=10,
    showgrid=False,
    gridwidth=1,
    gridcolor='grey',
    ticks='outside',
    zeroline=True,
    zerolinecolor='grey',
    zerolinewidth=1,
    showline=True,
    linewidth=2,
    linecolor='black'
)

fig.update_layout(
    xaxis_title="Time (min)",
    yaxis_title="Overpotential, η(mV) - iR<sub>u</sub>",
    font=dict(
        family="Times New Roman, monospace",
        size=18,
        color="dimgrey"
    )
)

# First, update layout for a transparent background and save the SVG
fig.update_layout({
    "plot_bgcolor": "rgba(0, 0, 0, 0)",
    "paper_bgcolor": "rgba(0, 0, 0, 0)"
})
fig.update_layout(width=500, height=500)
fig.write_image(f"{today_str} {fig_title} Chronopotentiometry.svg")

# Then update layout for a white background for display and JPEG saving
fig.update_layout({
    "plot_bgcolor": "rgba(255, 255, 255, 1)",
    "paper_bgcolor": "rgba(255, 255, 255, 1)"
})
fig.show()

fig.write_image(f"{today_str} {fig_title} Chronopotentiometry.jpeg")
